# Workflow Tracing Basics with OpenTelemetry

## Overview
This notebook demonstrates **built-in observability** in Agent Framework workflows using OpenTelemetry. You'll learn how to:
- Enable tracing with `setup_observability()`
- Understand automatic span generation for workflow lifecycle events
- Observe executor invocations and message passing
- Use environment variables to configure tracing, logging, and metrics
- Debug and monitor workflows in production

## What This Sample Does
A minimal two-executor workflow demonstrates automatic OpenTelemetry spans:
1. **StartExecutor** - Receives input, transforms to uppercase, sends message
2. **EndExecutor** - Receives message, prints final result

**Input:** `"hello tracing"`  
**Output:** `"HELLO TRACING"`  
**Tracing:** Automatic spans for build, run, executor execution, and message sending

## Key Concepts
- **OpenTelemetry (OTel):** Industry-standard observability framework
- **Spans:** Units of work that form distributed traces
- **Automatic Instrumentation:** Framework generates spans without manual code
- **Span Categories:** workflow.build, workflow.run, executor.process, message.send
- **Console Exporter:** Outputs traces to console for development

## Span Categories Generated

### 1. workflow.build
Emitted during workflow construction
- **Events:** build.started, build.validation_completed, build.completed, edge_group.process
- **Purpose:** Track workflow graph construction and validation

### 2. workflow.run
Emitted for each workflow execution
- **Events:** workflow.started, workflow.completed (or workflow.error)
- **Purpose:** Track end-to-end execution time and status

### 3. executor.process
Emitted for each executor invocation
- **Purpose:** Track individual executor execution time
- **Attributes:** executor_id, handler method

### 4. message.send
Emitted for each message sent between executors
- **Purpose:** Track message routing and data flow
- **Attributes:** source_id, target_id, message type

## Prerequisites

### Environment Variables (Optional)
Configure observability providers via environment variables:

**Tracing:**
```bash
ENABLE_OTEL=true                    # Enable OpenTelemetry tracing
OTEL_EXPORTER_TYPE=console          # Console exporter (dev)
OTEL_EXPORTER_TYPE=otlp             # OTLP exporter (production)
OTEL_EXPORTER_OTLP_ENDPOINT=...     # OTLP collector endpoint
```

**Logging:**
```bash
LOG_LEVEL=INFO                      # Set logging level
LOG_FORMAT=json                     # JSON structured logging
```

**Metrics:**
```bash
ENABLE_METRICS=true                 # Enable metrics collection
METRICS_EXPORTER_TYPE=console       # Console metrics exporter
```

### Default Behavior
If environment variables are not set, `setup_observability()` uses sensible defaults:
- Console exporter for development
- Standard logging format
- Basic metrics collection

## Step 1: Import Required Libraries

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

from agent_framework import Executor, WorkflowBuilder, WorkflowContext, get_logger, handler
from agent_framework.observability import setup_observability
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


## Step 2: Get Logger Instance
The framework provides a configured logger that integrates with OpenTelemetry.

In [ ]:
logger = get_logger()

## Step 3: Define Executor Classes

### StartExecutor
Transforms input to uppercase and forwards downstream.

**Spans Generated:**
- `executor.process` - Tracks handler execution time
- `message.send` - Tracks outbound message

### EndExecutor
Sink executor that prints the final result.

**Spans Generated:**
- `executor.process` - Tracks handler execution time

**Note:** The `# type: ignore[misc]` comments suppress mypy warnings related to the `@handler` decorator.

In [ ]:
class StartExecutor(Executor):
    @handler  # type: ignore[misc]
    async def handle_input(self, message: str, ctx: WorkflowContext[str]) -> None:
        # Transform and forward downstream. This produces executor.process and message.send spans.
        await ctx.send_message(message.upper())


class EndExecutor(Executor):
    @handler  # type: ignore[misc]
    async def handle_final(self, message: str, ctx: WorkflowContext) -> None:
        # Sink executor. The workflow completes when idle with no pending work.
        print(f"Final result: {message}")

## Step 4: Setup Observability

**What `setup_observability()` Does:**
1. Reads environment variables (ENABLE_OTEL, LOG_LEVEL, etc.)
2. Configures OpenTelemetry TracerProvider
3. Sets up span exporters (console, OTLP, etc.)
4. Initializes logging with trace correlation
5. Configures metrics providers

**Important:** Call this **before** building workflows to ensure all spans are captured!

In [ ]:
async def run_workflow():
    """Build and run the workflow with tracing enabled."""
    # This will enable tracing and create the necessary tracing, logging and metrics providers
    # based on environment variables.
    setup_observability()
    print("✓ Observability setup complete\n")

## Step 5: Build the Workflow

**Workflow Graph:**
```
Input → StartExecutor → EndExecutor → Output
```

**Spans Generated During Build:**
- `workflow.build` span wraps the entire build process
- Events: build.started → edge_group.process → build.validation_completed → build.completed

**Key Points:**
- `set_start_executor("start")` accepts executor ID string OR executor instance
- `build()` triggers workflow validation and span emission

In [ ]:
async def run_workflow():
    """Build and run the workflow with tracing enabled."""
    # Setup observability
    setup_observability()
    print("✓ Observability setup complete\n")
    
    # Build a two node graph: StartExecutor -> EndExecutor. The builder emits a workflow.build span.
    print("Building workflow...")
    workflow = (
        WorkflowBuilder()
        .add_edge(StartExecutor(id="start"), EndExecutor(id="end"))
        .set_start_executor("start")  # set_start_executor accepts an executor id string or the instance
        .build()
    )  # workflow.build span emitted here
    print("✓ Workflow built (workflow.build span emitted)\n")

## Step 6: Run the Workflow

**Spans Generated During Run:**

1. **workflow.run** - Overall execution span
   - Event: workflow.started
   - Child spans: executor.process (×2), message.send (×1)
   - Event: workflow.completed

2. **executor.process** (StartExecutor)
   - Executor: "start"
   - Handler: handle_input
   - Duration: Time to transform and send message

3. **message.send**
   - Source: "start"
   - Target: "end"
   - Message type: str
   - Payload: "HELLO TRACING"

4. **executor.process** (EndExecutor)
   - Executor: "end"
   - Handler: handle_final
   - Duration: Time to print result

**Trace Hierarchy:**
```
workflow.run
├── executor.process (start)
│   └── message.send (start → end)
└── executor.process (end)
```

In [ ]:
async def run_workflow():
    """Build and run the workflow with tracing enabled."""
    # Setup observability
    setup_observability()
    print("✓ Observability setup complete\n")
    
    # Build workflow
    print("Building workflow...")
    workflow = (
        WorkflowBuilder()
        .add_edge(StartExecutor(id="start"), EndExecutor(id="end"))
        .set_start_executor("start")
        .build()
    )
    print("✓ Workflow built (workflow.build span emitted)\n")
    
    # Run once with a simple payload. You should see workflow.run plus executor and message spans.
    print("Running workflow with input: 'hello tracing'")
    print("Expected spans: workflow.run + executor.process (×2) + message.send (×1)\n")
    
    await workflow.run("hello tracing")  # workflow.run + executor.process and message.send spans
    
    print("\n✓ Workflow completed (all spans emitted)")

## Step 7: Execute the Workflow

In [ ]:
await run_workflow()

## Expected Output

```
✓ Observability setup complete

Building workflow...
✓ Workflow built (workflow.build span emitted)

Running workflow with input: 'hello tracing'
Expected spans: workflow.run + executor.process (×2) + message.send (×1)

Final result: HELLO TRACING

✓ Workflow completed (all spans emitted)
```

### Console Exporter Output (Example)

When using console exporter (`OTEL_EXPORTER_TYPE=console`), you'll see span details:

```json
{
  "name": "workflow.build",
  "context": {
    "trace_id": "0x...",
    "span_id": "0x..."
  },
  "kind": "INTERNAL",
  "start_time": "2025-10-11T07:17:00.000000Z",
  "end_time": "2025-10-11T07:17:00.005000Z",
  "status": "OK",
  "events": [
    {"name": "build.started"},
    {"name": "edge_group.process"},
    {"name": "build.validation_completed"},
    {"name": "build.completed"}
  ]
}

{
  "name": "workflow.run",
  "context": {
    "trace_id": "0x...",
    "span_id": "0x..."
  },
  "kind": "INTERNAL",
  "start_time": "2025-10-11T07:17:00.010000Z",
  "end_time": "2025-10-11T07:17:00.050000Z",
  "status": "OK",
  "events": [
    {"name": "workflow.started"},
    {"name": "workflow.completed"}
  ]
}

{
  "name": "executor.process",
  "parent_id": "0x...",  // workflow.run span
  "attributes": {
    "executor_id": "start",
    "handler": "handle_input"
  },
  "start_time": "2025-10-11T07:17:00.015000Z",
  "end_time": "2025-10-11T07:17:00.020000Z",
  "status": "OK"
}

{
  "name": "message.send",
  "parent_id": "0x...",  // executor.process (start) span
  "attributes": {
    "source_id": "start",
    "target_id": "end",
    "message_type": "str"
  },
  "start_time": "2025-10-11T07:17:00.018000Z",
  "end_time": "2025-10-11T07:17:00.019000Z",
  "status": "OK"
}

{
  "name": "executor.process",
  "parent_id": "0x...",  // workflow.run span
  "attributes": {
    "executor_id": "end",
    "handler": "handle_final"
  },
  "start_time": "2025-10-11T07:17:00.025000Z",
  "end_time": "2025-10-11T07:17:00.030000Z",
  "status": "OK"
}
```

## Key Takeaways

### Automatic Instrumentation
✅ **Zero-Code Tracing:** No manual span creation required  
✅ **Comprehensive Coverage:** Build, run, executors, messages all traced  
✅ **Structured Events:** Rich metadata for debugging and analysis  
✅ **Performance Insights:** Execution times for every component

### Span Hierarchy
```
workflow.build
├── build.started (event)
├── edge_group.process (event)
├── build.validation_completed (event)
└── build.completed (event)

workflow.run
├── workflow.started (event)
├── executor.process (start)
│   └── message.send (start → end)
├── executor.process (end)
└── workflow.completed (event)
```

### Observability Benefits

**Development:**
- Debug execution flow and message routing
- Identify slow executors
- Validate workflow graph construction
- Console exporter for quick feedback

**Production:**
- Distributed tracing across services
- Performance monitoring and SLO tracking
- Error tracking with full context
- OTLP exporter to Jaeger, Zipkin, Azure Monitor, etc.

### Environment Configuration

**Development Setup:**
```bash
export ENABLE_OTEL=true
export OTEL_EXPORTER_TYPE=console
export LOG_LEVEL=DEBUG
```

**Production Setup (Azure Monitor):**
```bash
export ENABLE_OTEL=true
export OTEL_EXPORTER_TYPE=otlp
export OTEL_EXPORTER_OTLP_ENDPOINT=https://your-endpoint.azure.com
export OTEL_SERVICE_NAME=my-workflow-service
export LOG_LEVEL=INFO
export LOG_FORMAT=json
```

**Production Setup (Jaeger):**
```bash
export ENABLE_OTEL=true
export OTEL_EXPORTER_TYPE=otlp
export OTEL_EXPORTER_OTLP_ENDPOINT=http://jaeger:4317
export OTEL_SERVICE_NAME=my-workflow-service
```

## Understanding Span Attributes

### workflow.build Span
**Attributes:**
- `workflow.executor_count` - Number of executors in graph
- `workflow.edge_count` - Number of edges
- `workflow.start_executor_id` - Entry point executor

**Events:**
- `build.started` - Build process initiated
- `edge_group.process` - Processing edge definitions
- `build.validation_completed` - Graph validation passed
- `build.completed` - Build finished successfully

### workflow.run Span
**Attributes:**
- `workflow.run_id` - Unique run identifier
- `workflow.input_type` - Type of input message
- `workflow.status` - completed | error

**Events:**
- `workflow.started` - Execution started
- `workflow.completed` - Execution finished successfully
- `workflow.error` - Execution failed (with exception details)

### executor.process Span
**Attributes:**
- `executor_id` - Unique executor identifier
- `executor_type` - Class name (StartExecutor, EndExecutor)
- `handler_name` - Handler method name
- `message_type` - Type of input message
- `execution_time_ms` - Handler execution time

### message.send Span
**Attributes:**
- `source_id` - Sending executor ID
- `target_id` - Receiving executor ID (or "*" for broadcast)
- `message_type` - Message payload type
- `message_size` - Payload size in bytes (if applicable)

## Advanced Observability Patterns

### Custom Logging with Trace Correlation
```python
from agent_framework import get_logger

logger = get_logger()

class MyExecutor(Executor):
    @handler
    async def handle(self, msg: str, ctx: WorkflowContext[str]) -> None:
        # Logs are automatically correlated with current span
        logger.info("Processing message", extra={"message_length": len(msg)})
        
        result = msg.upper()
        
        logger.debug("Transformation complete", extra={"result": result})
        await ctx.send_message(result)
```

### Custom Span Events
```python
from opentelemetry import trace

class DataProcessorExecutor(Executor):
    @handler
    async def handle(self, data: dict, ctx: WorkflowContext) -> None:
        # Get current span
        span = trace.get_current_span()
        
        # Add custom event
        span.add_event("data.validation.started")
        
        # Validation logic
        if self._validate(data):
            span.add_event("data.validation.passed")
        else:
            span.add_event("data.validation.failed")
            span.set_status(trace.Status(trace.StatusCode.ERROR))
```

### Error Tracking
```python
class ErrorProneExecutor(Executor):
    @handler
    async def handle(self, msg: str, ctx: WorkflowContext) -> None:
        try:
            # Risky operation
            result = self._process(msg)
            await ctx.send_message(result)
        except Exception as e:
            # Exception is automatically recorded in span
            logger.error("Processing failed", exc_info=True)
            # Span status set to ERROR with exception details
            raise
```

## Integrating with Observability Platforms

### Azure Monitor (Application Insights)
```python
# Environment setup
import os
os.environ['ENABLE_OTEL'] = 'true'
os.environ['OTEL_EXPORTER_TYPE'] = 'azure-monitor'
os.environ['APPLICATIONINSIGHTS_CONNECTION_STRING'] = 'InstrumentationKey=...'

# Or use Azure SDK directly
from azure.monitor.opentelemetry import configure_azure_monitor
configure_azure_monitor()
```

### Jaeger
```python
import os
os.environ['ENABLE_OTEL'] = 'true'
os.environ['OTEL_EXPORTER_TYPE'] = 'otlp'
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = 'http://localhost:4317'
os.environ['OTEL_SERVICE_NAME'] = 'agent-framework-workflow'
```

### Datadog
```python
import os
os.environ['ENABLE_OTEL'] = 'true'
os.environ['OTEL_EXPORTER_TYPE'] = 'otlp'
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = 'http://localhost:4318'
os.environ['DD_SERVICE'] = 'agent-framework-workflow'
os.environ['DD_ENV'] = 'production'
```

### Local Development (Console)
```python
import os
os.environ['ENABLE_OTEL'] = 'true'
os.environ['OTEL_EXPORTER_TYPE'] = 'console'
os.environ['LOG_LEVEL'] = 'DEBUG'
```

## Performance Monitoring Queries

### Average Executor Execution Time
```sql
-- Example query for Azure Monitor
traces
| where operation_Name == "executor.process"
| summarize 
    avg(duration), 
    percentile(duration, 95), 
    percentile(duration, 99)
    by customDimensions.executor_id
| order by avg_duration desc
```

### Workflow Success Rate
```sql
traces
| where operation_Name == "workflow.run"
| summarize 
    total=count(),
    successful=countif(customDimensions.status == "completed"),
    failed=countif(customDimensions.status == "error")
| project 
    total,
    successful,
    failed,
    success_rate=100.0 * successful / total
```

### Message Flow Analysis
```sql
traces
| where operation_Name == "message.send"
| summarize count() by 
    source=customDimensions.source_id,
    target=customDimensions.target_id
| order by count_ desc
```

## Best Practices

### Do's
✅ **Call `setup_observability()` early** - Before building workflows  
✅ **Use meaningful executor IDs** - Makes trace analysis easier  
✅ **Add custom events** for critical operations  
✅ **Use structured logging** with trace correlation  
✅ **Set service name** in production (`OTEL_SERVICE_NAME`)  
✅ **Monitor span duration** to identify bottlenecks  
✅ **Export to OTLP** in production for centralized analysis

### Don'ts
❌ **Don't log sensitive data** in span attributes  
❌ **Don't create excessive custom spans** (use events instead)  
❌ **Don't ignore error spans** - they indicate reliability issues  
❌ **Don't use console exporter** in production  
❌ **Don't disable tracing** without good reason  
❌ **Don't forget to set environment** (dev/staging/prod)

### Production Checklist
- [ ] Environment variables configured (`ENABLE_OTEL=true`)
- [ ] OTLP endpoint set and accessible
- [ ] Service name meaningful and unique
- [ ] Log level appropriate (INFO or WARNING)
- [ ] Sampling configured if high volume (not shown in sample)
- [ ] Metrics enabled for dashboards
- [ ] Alerts configured for error spans
- [ ] Trace retention policies set

## Troubleshooting

### No Spans Appearing
**Problem:** `setup_observability()` not called or called after workflow build

**Solution:**
```python
# CORRECT order
setup_observability()  # First!
workflow = WorkflowBuilder().build()  # Then build
await workflow.run(input)  # Then run
```

### Spans Not Exported
**Problem:** Exporter not configured or endpoint unreachable

**Solution:**
```python
# Check environment
import os
print(os.environ.get('ENABLE_OTEL'))  # Should be 'true'
print(os.environ.get('OTEL_EXPORTER_TYPE'))  # Should be set
print(os.environ.get('OTEL_EXPORTER_OTLP_ENDPOINT'))  # Should be reachable
```

### Missing Executor Spans
**Problem:** Executor IDs not set, causing span attribute issues

**Solution:**
```python
# Always provide executor IDs
StartExecutor(id="start")  # Good
StartExecutor()  # Bad - auto-generated ID is less useful
```

### High Overhead
**Problem:** Too many spans or large attributes

**Solution:**
```python
# Configure sampling (advanced)
from opentelemetry.sdk.trace.sampling import TraceIdRatioBased

# Sample 10% of traces in high-volume scenarios
os.environ['OTEL_TRACES_SAMPLER'] = 'traceidratio'
os.environ['OTEL_TRACES_SAMPLER_ARG'] = '0.1'
```

## Next Steps

### Learn More About Workflows
- Explore `control-flow/` notebooks for complex routing patterns
- See `orchestration/` for multi-agent coordination
- Check `checkpoint/` for stateful workflows with observability

### Advanced Observability
- Custom metrics with OpenTelemetry Metrics API
- Distributed tracing across microservices
- Profiling executor performance
- Real-time alerting on error spans

### OpenTelemetry Resources
- [OpenTelemetry Documentation](https://opentelemetry.io/docs/)
- [OpenTelemetry Python SDK](https://opentelemetry-python.readthedocs.io/)
- [Azure Monitor Integration](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-enable)
- [Jaeger Tracing](https://www.jaegertracing.io/docs/)